In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install transformers
!pip install nltk


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, AdamW
import os
import re
import nltk
from torch.utils.data import DataLoader
import torch
from torch.nn import functional as F



# Tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
# model.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


# Preprocessing function
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing punctuation
    text = re.sub(r'[^\w\s.]', '', text)

     # removing digits
    text = re.sub(r'\d', '', text)

    # Removing URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Removing stopwords
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    text = ' '.join([token for token in tokens if token not in stop_words])
    # Remove extra spaces
    text = re.sub(' +', ' ', text)

    # Remove extra spaces
    text = re.sub(' +', ' ', text)
    return text




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob
import os
from tqdm import tqdm

def parse_data(data_dir, gold_dir):
    articles=[]
    golden_summaries=[]
    os.chdir(data_dir)
    for file_name in tqdm(glob.glob("*.txt")):
        file_path=os.path.join(data_dir, file_name)
        article = open(file_path,encoding="utf-8").read()
        # if file_name=="17.txt":
        #   print(article)
        for i in [1, 2, 3, 4, 5, 6, 7, 8]:
            try:
                name, extension = file_name.split(".")
                summary = open(os.path.join(gold_dir, name + "_" + str(i) + "." + extension),encoding="utf-8").read()
                golden_summaries.append(summary)
                article=preprocess_text(article)
                articles.append(article)

            except Exception as e:
                continue
    return articles, golden_summaries

train_articles,train_summaries = parse_data("/content/drive/MyDrive/ProjectNLP/Annual_reports", "/content/drive/MyDrive/ProjectNLP/gold_summaries")
# valid_articles,valid_summaries = parse_data("path for valid .txt files", "path for validation gold summaries")


100%|██████████| 9/9 [00:17<00:00,  1.91s/it]


In [ ]:
import glob
import os
from tqdm import tqdm
from transformers import BartTokenizer
from torch.utils.data import Dataset
from transformers import BartForConditionalGeneration, BartTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader



class CustomDataset(Dataset):
    def __init__(self, articles, summaries, tokenizer):
        self.articles = articles
        self.summaries = summaries
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.articles)

    def __getitem__(self, idx):
        article = self.articles[idx]
        summary = self.summaries[idx]

        # Tokenize article and summary
        tokenized_inputs = self.tokenizer(article, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
        tokenized_labels = self.tokenizer(summary, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

        return {
            'input_ids': tokenized_inputs['input_ids'].squeeze(),
            'labels': tokenized_labels['input_ids'].squeeze()
        }



# Parse data
# train_articles, train_summaries = parse_data("/content/drive/MyDrive/ProjectNLP/Annual_reports", "/content/drive/MyDrive/ProjectNLP/gold_summaries")

# Create CustomDataset
train_dataset = CustomDataset(train_articles, train_summaries, tokenizer)
num_epochs=2
batch_size = 2
# Training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Bart_model",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    save_steps=1000,  # Number of steps between model checkpoints
     save_total_limit=5,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,

    data_collator=None
)

# Training loop using Trainer
trainer.train()


In [ ]:


# Save the model configuration

trainer.save_model("/content/drive/MyDrive/Bart_model")


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the trained model
model_path = "/content/drive/MyDrive/Bart_model"
trained_model = BartForConditionalGeneration.from_pretrained(model_path)

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/Bart_model'. Use `repo_type` argument if needed.

In [ ]:


from transformers import AutoTokenizer, AutoModelForCausalLM


new_report_path = "/content/drive/MyDrive/Nlp_validation/Annual_Report/30830.txt"
new_report_content = open(new_report_path, 'r', encoding='utf-8').read().strip()
new_report_content = preprocess_text(new_report_content)

len(new_report_content)


243877

In [ ]:

sentences = nltk.tokenize.sent_tokenize(new_report_content)

In [ ]:
len(sentences)

6870

In [ ]:
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

694

In [ ]:
length =0
chunk = ""
chunks = []
count = -1

for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length

  if combined_length <= 512:
    chunk += sentence + " "
    length = combined_length

    if count == len(sentences) - 1:
      chunks.append(chunk.strip())

  else:
    chunks.append(chunk.strip())

    length = 0
    chunk = ""

    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))

len(chunks)

74

In [ ]:
[len(tokenizer.tokenize(c)) for c in chunks]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model.to(device)

NameError: name 'trained_model' is not defined

In [ ]:
inputs = [tokenizer(chunk , return_tensors="pt", max_length=512, truncation=True, padding='max_length') for chunk in chunks]

In [ ]:
chunk_summaries = []
for input in inputs:
  output = model.generate(**input)
  generated_summary = tokenizer.decode(output[0], skip_special_tokens=True)
  chunk_summaries.append(generated_summary)

final_summary = ' '.join(chunk_summaries)
print(final_summary)

In [ ]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output , skip_special_token=True))
chunk_summaries = []
for input in inputs:
    # Ensure input tensor is on the same device as the model
    input = {key: value.to(device) for key, value in input.items()}
    output = trained_model.generate(**input)

    generated_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    chunk_summaries.append(generated_summary)

final_summary = '. '.join(chunk_summaries)

output_file_path = "/content/drive/MyDrive/Final_summary/final_summary.txt"
with open(output_file_path, "w", encoding="utf-8") as output_file:
    output_file.write(final_summary)
print("generated summary", final_summary)

In [ ]:
def summarize_report(report_path):
    report_content = open(report_path, 'r', encoding='utf-8').read().strip()
    report_content = preprocess_text(report_content)

    sentences = nltk.tokenize.sent_tokenize(report_content)
    length = 0
    chunk = ""
    chunks = []
    count = -1

    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length

        if combined_length <= 1022:
            chunk += sentence + " "
            length = combined_length

            if count == len(sentences) - 1:
                chunks.append(chunk.strip())
        else:
            chunks.append(chunk.strip())
            length = 0
            chunk = ""
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trained_model.to(device)
    inputs = [tokenizer(chunk, return_tensors="pt", max_length=1024, truncation=True, padding='max_length') for chunk in chunks]

    chunk_summaries = []
    for input in inputs:
        # Ensure input tensor is on the same device as the model
        input = {key: value.to(device) for key, value in input.items()}
        output = trained_model.generate(**input)

        generated_summary = tokenizer.decode(output[0], skip_special_tokens=True)
        chunk_summaries.append(generated_summary)

    final_summary = ' '.join(chunk_summaries)
    return final_summary

# Path to the folder containing annual reports
reports_folder = "/content/drive/MyDrive/Datasets/Annual_Report/"
# Path to the target folder to save generated summaries
target_folder = "/content/drive/MyDrive/Datasets_summary/"

# Iterate through all files in the reports folder
for filename in os.listdir(reports_folder):
    if filename.endswith(".txt"):
        report_path = os.path.join(reports_folder, filename)
        generated_summary = summarize_report(report_path)

        # Save the generated summary to the target folder with the same name as the input report
        output_file_path = os.path.join(target_folder, f"{filename.split('.')[0]}_{index}.txt")
        with open(output_file_path, "w", encoding="utf-8") as output_file:
            output_file.write(generated_summary)

        print(f"Generated summary for {filename}")

print("All reports summarized and saved.")

In [ ]:
pip install rouge

In [ ]:
from rouge import Rouge

def calculate_rouge2_score(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)
    rouge2_score = scores[0]['rouge-2']['f']

    return rouge2_score



scores=[]
file_name="65.txt"
for i in [1, 2, 3, 4, 5, 6, 7, 8]:
    try:
        name, extension = file_name.split(".")
        test_summary = open(os.path.join('/content/drive/MyDrive/Datasets/gold_summary', name + "_" + str(i) + "." + extension),encoding="utf-8").read()
        rouge2_score = calculate_rouge2_score(test_summary, final_summary)
        scores.append(rouge2_score)
    except Exception as e:
        continue

scores

In [ ]:

from rouge import Rouge

def calculate_rouge_scores(reference_path, generated_summary_path):
    with open(reference_path, 'r', encoding='utf-8') as ref_file:
        reference_text = ref_file.read()

    with open(generated_summary_path, 'r', encoding='utf-8') as gen_file:
        generated_summary_text = gen_file.read()

    rouge = Rouge()
    scores = rouge.get_scores(generated_summary_text, reference_text)

    return scores

# Example usage:
reference_path = '/content/drive/MyDrive/30779_1.txt'  # Replace with the path to your reference summary
generated_summary_path = '/content/drive/MyDrive/BART_lexrank/output/rank_summary.txt'  # Replace with the path to your generated summary

rouge_scores = calculate_rouge_scores(reference_path, generated_summary_path)
print("ROUGE-1:", rouge_scores[0]['rouge-1'])
print("ROUGE-2:", rouge_scores[0]['rouge-2'])
print("ROUGE-L:", rouge_scores[0]['rouge-l'])


ROUGE-1: {'r': 0.10469591993841416, 'p': 0.30022075055187636, 'f': 0.1552511377183611}
ROUGE-2: {'r': 0.0030873726458783574, 'p': 0.013850415512465374, 'f': 0.005049227011383433}
ROUGE-L: {'r': 0.10238645111624327, 'p': 0.293598233995585, 'f': 0.15182648018411454}
